# Loop 11 Analysis: Strategy Assessment

## Key Observations
1. 11 experiments with only 0.000521 total improvement (0.0007%)
2. 9 consecutive zero-improvement results
3. Gap to target: 1.74 points (2.5%)
4. All optimization approaches exhausted

## Critical Insight from Web Search
Top teams use **hybrid strategies**:
- N < 58: Simulated Annealing for chaotic packings
- N >= 58: Crystalline/Lattice packing (mathematically superior for large N)

## What We've Tried vs What We Haven't
### Tried:
- SA optimization (multiple variants)
- Fractional translation
- Random restart
- Asymmetric search
- NFP construction
- Ensemble of public CSVs
- Rebuild from corners

### NOT Tried:
- True crystalline lattice packing for large N
- Double-lattice (Kuperberg) construction
- Strip packing then square conversion

In [ ]:
# Let's analyze the score distribution by N range to understand where improvements might be possible
import pandas as pd
import numpy as np

# Load current submission
df = pd.read_csv('/home/submission/submission.csv')
df['n'] = df['id'].str[:3].astype(int)
df['idx'] = df['id'].str[4:].astype(int)

# Parse values
def parse_val(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

df['x'] = df['x'].apply(parse_val)
df['y'] = df['y'].apply(parse_val)
df['deg'] = df['deg'].apply(parse_val)

print(f"Total rows: {len(df)}")
print(f"N range: {df['n'].min()} to {df['n'].max()}")
print(f"Unique N values: {df['n'].nunique()}")
print()

In [ ]:
# Calculate score per N
import math

TX = [0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125]
TY = [0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5]

def get_bbox(group):
    xmin, xmax = float('inf'), float('-inf')
    ymin, ymax = float('inf'), float('-inf')
    
    for _, row in group.iterrows():
        cx, cy, deg = row['x'], row['y'], row['deg']
        rad = deg * math.pi / 180
        c, s = math.cos(rad), math.sin(rad)
        
        for tx, ty in zip(TX, TY):
            px = tx * c - ty * s + cx
            py = tx * s + ty * c + cy
            xmin = min(xmin, px)
            xmax = max(xmax, px)
            ymin = min(ymin, py)
            ymax = max(ymax, py)
    
    return max(xmax - xmin, ymax - ymin)

scores = []
for n in range(1, 201):
    group = df[df['n'] == n]
    side = get_bbox(group)
    score = side * side / n
    scores.append({'n': n, 'side': side, 'score': score})

scores_df = pd.DataFrame(scores)
print(f"Total score: {scores_df['score'].sum():.6f}")
print(f"Target: 68.919154")
print(f"Gap: {scores_df['score'].sum() - 68.919154:.6f}")
print()

In [ ]:
# Analyze score contribution by N range
ranges = [(1, 10), (11, 25), (26, 50), (51, 100), (101, 150), (151, 200)]

print("Score contribution by N range:")
print("="*60)
for start, end in ranges:
    mask = (scores_df['n'] >= start) & (scores_df['n'] <= end)
    range_score = scores_df[mask]['score'].sum()
    pct = range_score / scores_df['score'].sum() * 100
    print(f"N={start:3d}-{end:3d}: {range_score:8.4f} ({pct:5.2f}%)")

print("="*60)
print(f"Total: {scores_df['score'].sum():.6f}")
print()

In [ ]:
# Analyze score contribution by N range\nranges = [(1, 10), (11, 25), (26, 50), (51, 100), (101, 150), (151, 200)]\n\nprint("Score contribution by N range:")\nprint("="*60)\nfor start, end in ranges:\n    mask = (scores_df['n'] >= start) & (scores_df['n'] <= end)\n    range_score = scores_df[mask]['score'].sum()\n    pct = range_score / scores_df['score'].sum() * 100\n    print(f"N={start}-{end}: {range_score:.4f} ({pct:.2f}%)")\n\nprint("="*60)\nprint(f"Total: {scores_df['score'].sum():.6f}")\nprint()

In [ ]:
# Calculate packing efficiency for each N\n# Tree area = 0.24562 (from previous analysis)\nTREE_AREA = 0.24562\n\nscores_df['packing_efficiency'] = (scores_df['n'] * TREE_AREA) / (scores_df['side'] ** 2) * 100\n\nprint("Packing efficiency by N range:")\nprint("="*60)\nfor start, end in ranges:\n    mask = (scores_df['n'] >= start) & (scores_df['n'] <= end)\n    avg_eff = scores_df[mask]['packing_efficiency'].mean()\n    min_eff = scores_df[mask]['packing_efficiency'].min()\n    max_eff = scores_df[mask]['packing_efficiency'].max()\n    print(f"N={start}-{end}: avg={avg_eff:.2f}%, min={min_eff:.2f}%, max={max_eff:.2f}%")\n\nprint()

In [ ]:
# Find N values with lowest packing efficiency (most room for improvement)\nprint("N values with LOWEST packing efficiency (most room for improvement):")\nprint("="*60)\nlowest_eff = scores_df.nsmallest(20, 'packing_efficiency')\nfor _, row in lowest_eff.iterrows():\n    n = int(row['n'])\n    print(f"N={n}: efficiency={row['packing_efficiency']:.2f}%, side={row['side']:.6f}, score={row['score']:.6f}")\nprint()

In [ ]:
# Check if there are any N values where different public CSVs have different topologies
# This would indicate room for improvement

import os
import glob

# Find all CSV files in snapshots
csv_files = glob.glob('/home/nonroot/snapshots/santa-2025/*/submission/*.csv')
print(f"Found {len(csv_files)} CSV files in snapshots")

# Also check research/kernels
kernel_csvs = glob.glob('/home/code/research/kernels/*/*.csv')
print(f"Found {len(kernel_csvs)} CSV files in kernels")

# Check data directory
data_csvs = glob.glob('/home/data/*.csv')
print(f"Found {len(data_csvs)} CSV files in data")
print()

In [ ]:
# Check if there are any N values where different public CSVs have different topologies\n# This would indicate room for improvement\n\nimport os\nimport glob\n\n# Find all CSV files in snapshots\ncsv_files = glob.glob('/home/nonroot/snapshots/santa-2025/*/submission/*.csv')\nprint(f"Found {len(csv_files)} CSV files in snapshots")\n\n# Also check research/kernels\nkernel_csvs = glob.glob('/home/code/research/kernels/*/*.csv')\nprint(f"Found {len(kernel_csvs)} CSV files in kernels")\n\n# Check data directory\ndata_csvs = glob.glob('/home/data/*.csv')\nprint(f"Found {len(data_csvs)} CSV files in data")\nprint()

In [ ]:
# Let's look at the angle distribution in our current solution\n# This tells us about the topology\n\nprint("Angle distribution in current solution:")\nprint("="*60)\n\nfor n in [1, 2, 5, 10, 20, 50, 100, 150, 200]:\n    group = df[df['n'] == n]\n    angles = group['deg'].values\n    unique_angles = np.unique(np.round(angles, 1))\n    print(f"N={n}: {len(unique_angles)} unique angles")\n    if len(unique_angles) <= 5:\n        print(f"        Angles: {unique_angles}")\n    else:\n        print(f"        Sample: {unique_angles[:5]}...")\nprint()

## Key Findings

1. **Score Distribution**: The gap is distributed across ALL N values, not concentrated in any range
2. **Packing Efficiency**: Large N values have 86-88% efficiency, which is very high
3. **Required Improvement**: Need ~1.24% uniform side reduction - extremely tight
4. **Angle Patterns**: Current solutions use complex angle configurations, not simple lattice patterns

## Critical Insight

The web search revealed that top teams achieving scores below 69 use:
1. **Hybrid strategies**: SA for N < 58, crystalline packing for N >= 58
2. **Asymmetric layouts**: Not enforcing square symmetry
3. **Strip packing**: Optimize for rectangular strip first, then adjust

## What We Should Try Next

1. **True Double-Lattice Construction**: Use Kuperberg's method with proper lattice vectors
2. **Strip Packing First**: Optimize for minimum width strip, then convert to square
3. **Genetic Algorithm with Topology Crossover**: Exchange tree arrangements between solutions
4. **Focus on Specific N Values**: The lowest efficiency N values have most room for improvement